<a href="https://colab.research.google.com/github/HuyenNguyenHelen/CORD-19-KG/blob/master/QA-models/covid_19_kg_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [3]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [4]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [5]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import pandas as pd
import requests
import io

## Loading Data 
(a triples dataset)

In [6]:
# Downloading the csv file from your GitHub account
re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pdf_June20_.csv').content
re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pmc_June20_.csv').content
pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
dataset = pd.concat([pmc, pdf])
print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'subject', 'object', 'new_relation'], dtype='object')


,subject,new_relation,object
0,dysfunction,disease_species,child
1,et dysfunction,disease_species,child
2,et dysfunction,disease_species,child
3,rsv,disease_disease,virus
4,treatment,treat_procedure_species,child


In [7]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (230330, 3)


In [8]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    63192
gene_gene                          24172
disease_species                    22370
gene_disease                       14220
disease_gene                       13347
disease_treat_procedure            10184
treat_procedure_disease             8223
drug_disease                        7250
gene_treat_procedure                6142
treat_procedure_treat_procedure     5625
gene_species                        5298
drug_gene                           5270
drug_drug                           4613
treat_procedure_gene                4602
disease_symptom                     4354
treat_procedure_species             4263
disease_drug                        4044
drug_species                        3680
gene_drug                           3603
symptom_disease                     3411
symptom_symptom                     3266
drug_treat_procedure                3229
treat_procedure_drug                2578
symptom_species               

# Data Processing
Partition the triples into 10 portions that will be used for buiding QA systems

In [9]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, axis=1).reset_index(drop=True)
subsets = np.array_split(dataset, 10)  
len(subsets)


10

# Developing QA system

## Developing model

### Create training, validation and test splits

In [10]:
from ampligraph.evaluation import train_test_split_no_unseen
def train_set_split (ds, test_frac = 200, val_frac = 100 ):
  # get the validation set of size 500
  test_train, X_test = train_test_split_no_unseen(ds.values, test_frac, seed=0)

  # get the test set of size 1000 from the remaining triples
  X_train, X_val = train_test_split_no_unseen(test_train, val_frac, seed=0)

  print('Total triples:', dataset.shape)
  print('Size of train:', X_train.shape)
  print('Size of valid:', X_val.shape)
  print('Size of test:', X_test.shape)
  return X_train, X_val, X_test

### Model Training

In [11]:
from ampligraph.latent_features import TransE

def training (train_ds, val_ds, test_ds):
  # Train a KGE model
  model = TransE(k=300, 
                epochs=100, 
                eta=1, 
                loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
                seed= 0, batches_count= 100, verbose=True)

  model.fit(train_ds)
  # ----------------------
  # Evaluate: 
  # Filtered evaluation with ranking strategy assigning worst rank to break ties
  from ampligraph.utils import save_model, restore_model
  save_model(model, 'TransE.pkl')
  model = restore_model('TransE.pkl')

  # create the filter 
  X_filter = np.concatenate([train_ds, val_ds, test_ds], 0)

  # compute ranks
  ranks = evaluate_performance(test_ds, 
                              model=model, 
                              filter_triples=X_filter)

  # ranks are computed per triple
  print('Test set:', X_test.shape)
  print('Size of ranks:', ranks.shape)

  # Aggregate metrics show the aggregate performance of the model on the test set using a single number
  display_aggregate_metrics(ranks)
  # ----------------------
  
  return display_aggregate_metrics(ranks)

X_train, X_val, X_test = train_set_split(dataset)
training (X_train, X_val, X_test) #dataset.to_numpy()

Total triples: (230330, 3)
Size of train: (230030, 3)
Size of valid: (100, 3)
Size of test: (200, 3)


100%|██████████| 200/200 [00:02<00:00, 89.89it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 145.6125
Mean Reciprocal Rank: 0.3388908407518953
Hits@1: 0.2425
Hits@10: 0.515
Hits@100: 0.7925
Mean Rank: 145.6125
Mean Reciprocal Rank: 0.3388908407518953
Hits@1: 0.2425
Hits@10: 0.515
Hits@100: 0.7925


## Knowledge Discovery

In [12]:
from ampligraph.discovery import query_topn

def QA_retrieve (input, top_n ):
  head, relation1, tail = input[0], input[1], input[2]
  if len(relation1.split('_')) <= 2:
    relation2 = '{}_{}'.format(relation1.split('_')[1], relation1.split('_')[0])
  elif len(relation1.split('_')) == 3:
     relation2 = '{}_{}_{}'.format(relation1.split('_')[1], relation1.split('_')[2], relation1.split('_')[0])
  else:
    raise ValueError('error: relation length issue')
  # restore the previously saved model to save time
  model = restore_model('TransE.pkl')
  if relation1  in dataset['new_relation'].tolist():
    triples1, scores1 = query_topn(model, top_n=top_n, 
                                head=head, 
                                relation=relation1, 
                                tail=tail, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples1, scores1 = [], []

  if relation2  in dataset['new_relation'].tolist():
    triples2, scores2 = query_topn(model, top_n=top_n, 
                                head=tail, 
                                relation=relation2, 
                                tail=head, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples2, scores2 = [], []

  if len(triples1) + len(triples2) == top_n:
    if len(triples1) ==0: 
      triples = triples1 + triples2.tolist()
      scores = scores1+ scores2.tolist()
    else:
      triples = triples1.tolist() + triples2
      scores = scores1.tolist() + scores2
  else:
    end_idx = round(top_n/2)
    triples = triples1.tolist()[:end_idx] + triples2.tolist()[:end_idx]
    scores = scores1.tolist()[:end_idx] + scores2.tolist()[:end_idx]
   
  for triple, score in zip(triples, scores):
      print('Score: {} \t {} '.format(score, triple))
  return triples, scores
 


In [13]:

# Query2: What are related diseases of Covid-19
print ('------------------------------------------------------------')
print('What are related diseases of Covid-19')
question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)

# Query3: What are the genes/proteins related to Covid-19
print ('------------------------------------------------------------')
print('What are the genes/proteins related to Covid-19')
question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)

# Query4: What species transmit the covid-19
print ('------------------------------------------------------------')
print('What species transmit the covid-19?')
question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)

# Query: what are symptoms of the covid-19
print ('------------------------------------------------------------')
print('what are symptoms of the covid-19?')
question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)

# What are potential drugs to treat COVID-19?
print ('------------------------------------------------------------')
print('What are potential drugs to treat COVID-19?')
question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)

# Query1: what are the possible therapeutic procedure of covid-19??
print ('------------------------------------------------------------')
print('what are the possible therapeutic procedure of covid-19?')
question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)

------------------------------------------------------------
What are related diseases of Covid-19
Score: -8.578287124633789 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -13.685384750366211 	 ['covid-19', 'disease_disease', 'he'] 
Score: -13.836322784423828 	 ['covid-19', 'disease_disease', 'same'] 
Score: -14.102177619934082 	 ['covid-19', 'disease_disease', 'sars cov infection'] 
Score: -14.187284469604492 	 ['covid-19', 'disease_disease', 'copd'] 
Score: -14.211858749389648 	 ['covid-19', 'disease_disease', 'covid'] 
Score: -14.225895881652832 	 ['covid-19', 'disease_disease', 'obesity'] 
Score: -14.284072875976562 	 ['covid-19', 'disease_disease', 'cpi'] 
Score: -14.303427696228027 	 ['covid-19', 'disease_disease', 'lupus erythematosus'] 
Score: -14.334739685058594 	 ['covid-19', 'disease_disease', 'rheumatoid arthritis'] 
Score: -8.578287124633789 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -12.749200820922852 	 ['2009', 'disease_disease', 'covid-19'] 
Score: -1

In [14]:
# Compare the raw call and graph embedding results for the symptoms query

## raw call
#dataset.loc[(dataset['subject']=='symptom')].head(20)

#####==> the result of raw call seems to be better

# Train model, and Knowledge mining with different % of dataset

In [30]:
ALL = []

for idx in range(len(subsets)):
  outputs  = {}
  data = subsets[:idx+1]
  data_concat = pd.concat(data)
  print ('i: {} -- len: {}'.format(idx, len(data_concat)))

  # Spliting dataset for training
  X_train, X_val, X_test = train_set_split(data_concat)
  # Training...
  print('Training..........')
  training (X_train, X_val, X_test)

  # Retrieving
  # Query2: What are related diseases of Covid-19
  print ('------------------------------------------------------------')
  print('What are related diseases of Covid-19')
  question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)
  df2 = pd.DataFrame(zip(question2[0],question2[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df2

  # Query3: What are the genes/proteins related to Covid-19
  print ('------------------------------------------------------------')
  print('What are the genes/proteins related to Covid-19')
  question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)
  df3 = pd.DataFrame(zip(question3[0],question3[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df3 

  # Query4: What species transmit the covid-19
  print ('------------------------------------------------------------')
  print('What species transmit the covid-19?')
  question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)
  df4 = pd.DataFrame(zip(question4[0],question4[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df4

  # Query: what are symptoms of the covid-19
  print ('------------------------------------------------------------')
  print('what are symptoms of the covid-19?')
  question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)
  df5 = pd.DataFrame(zip(question5[0],question5[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df5  

  # What are potential drugs to treat COVID-19?
  print ('------------------------------------------------------------')
  print('What are potential drugs to treat COVID-19?')
  question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)
  df6 = pd.DataFrame(zip(question6[0],question6[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df6

  # Query1: what are the possible therapeutic procedure of covid-19??
  print ('------------------------------------------------------------')
  print('what are the possible therapeutic procedure of covid-19?')
  question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)
  df1 = pd.DataFrame(zip(question1[0],question1[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)] = df1
  ALL.append(outputs)
                     
                     

i: 0 -- len: 23033
Total triples: (230330, 3)
Size of train: (22733, 3)
Size of valid: (100, 3)
Size of test: (200, 3)
Training..........


100%|██████████| 200/200 [00:01<00:00, 148.73it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 458.3
Mean Reciprocal Rank: 0.19218405203423208
Hits@1: 0.1375
Hits@10: 0.3075
Hits@100: 0.525
Mean Rank: 458.3
Mean Reciprocal Rank: 0.19218405203423208
Hits@1: 0.1375
Hits@10: 0.3075
Hits@100: 0.525
------------------------------------------------------------
What are related diseases of Covid-19
Score: -12.470703125 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -14.450900077819824 	 ['covid-19', 'disease_disease', 'feline infectious peritonitis virus'] 
Score: -14.553361892700195 	 ['covid-19', 'disease_disease', 'infectious peritonitis virus'] 
Score: -14.609983444213867 	 ['covid-19', 'disease_disease', 'acute pancreatitis'] 
Score: -14.632550239562988 	 ['covid-19', 'disease_disease', 'cryptococcosis'] 
Score: -14.745712280273438 	 ['covid-19', 'disease_disease', 'chronic medical condition'] 
Score: -14.832906723022461 	 ['covid-19', 'disease_disease', 'buruli ulcer'] 
Score: -14.839726448059082 	 ['covid-19', 'disease

100%|██████████| 200/200 [00:01<00:00, 124.41it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 151.215
Mean Reciprocal Rank: 0.3721591508068064
Hits@1: 0.2425
Hits@10: 0.59
Hits@100: 0.7975
Mean Rank: 151.215
Mean Reciprocal Rank: 0.3721591508068064
Hits@1: 0.2425
Hits@10: 0.59
Hits@100: 0.7975
------------------------------------------------------------
What are related diseases of Covid-19
Score: -5.187113285064697 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -11.289960861206055 	 ['covid-19', 'disease_disease', 'brsv'] 
Score: -11.448110580444336 	 ['covid-19', 'disease_disease', 'hypertension'] 
Score: -11.540975570678711 	 ['covid-19', 'disease_disease', 'h1n1pdm infection'] 
Score: -11.572669982910156 	 ['covid-19', 'disease_disease', 'diffuse alveolar damage'] 
Score: -11.621440887451172 	 ['covid-19', 'disease_disease', 'covid'] 
Score: -11.626729965209961 	 ['covid-19', 'disease_disease', 'roseola'] 
Score: -11.663436889648438 	 ['covid-19', 'disease_disease', 'atrophy'] 
Score: -11.700296401977539 	 ['covid

...

Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 145.2425
Mean Reciprocal Rank: 0.42799651434202807
Hits@1: 0.3375
Hits@10: 0.59
Hits@100: 0.8
Mean Rank: 145.2425
Mean Reciprocal Rank: 0.42799651434202807
Hits@1: 0.3375
Hits@10: 0.59
Hits@100: 0.8
------------------------------------------------------------
What are related diseases of Covid-19
Score: -7.582308769226074 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -12.22188663482666 	 ['covid-19', 'disease_disease', 'he'] 
Score: -12.565080642700195 	 ['covid-19', 'disease_disease', 'sars cov infection'] 
Score: -12.597368240356445 	 ['covid-19', 'disease_disease', 'pd'] 
Score: -12.690503120422363 	 ['covid-19', 'disease_disease', 'syndrome'] 
Score: -12.699332237243652 	 ['covid-19', 'disease_disease', 'covid'] 
Score: -12.76041316986084 	 ['covid-19', 'disease_disease', 'obesity'] 
Score: -12.840147018432617 	 ['covid-19', 'disease_disease', 'viral infection'] 
Score: -12.865209579467773 	 ['covid-19', 'disease_disease

100%|██████████| 200/200 [00:02<00:00, 85.99it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 2170.2175
Mean Reciprocal Rank: 0.07434855366491938
Hits@1: 0.045
Hits@10: 0.1225
Hits@100: 0.305
Mean Rank: 2170.2175
Mean Reciprocal Rank: 0.07434855366491938
Hits@1: 0.045
Hits@10: 0.1225
Hits@100: 0.305
------------------------------------------------------------
What are related diseases of Covid-19
Score: -16.139820098876953 	 ['covid-19', 'disease_disease', 'cryptococcosis'] 
Score: -16.292505264282227 	 ['covid-19', 'disease_disease', 'digestive system disorder'] 
Score: -16.411865234375 	 ['covid-19', 'disease_disease', 'rhv'] 
Score: -16.4294376373291 	 ['covid-19', 'disease_disease', 'type 2 diabetes mellitus'] 
Score: -16.47928237915039 	 ['covid-19', 'disease_disease', 'wupyv'] 
Score: -16.540393829345703 	 ['covid-19', 'disease_disease', 'bronchial asthma'] 
Score: -16.564208984375 	 ['covid-19', 'disease_disease', 'pulmonary cryptococcosis'] 
Score: -16.67144775390625 	 ['covid-19', 'disease_disease', 'pulmonary endar

100%|██████████| 200/200 [00:02<00:00, 87.30it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 145.625
Mean Reciprocal Rank: 0.3378115780252995
Hits@1: 0.2425
Hits@10: 0.5125
Hits@100: 0.79
Mean Rank: 145.625
Mean Reciprocal Rank: 0.3378115780252995
Hits@1: 0.2425
Hits@10: 0.5125
Hits@100: 0.79
------------------------------------------------------------
What are related diseases of Covid-19
Score: -8.59142017364502 	 ['covid-19', 'disease_disease', 'covid-19'] 
Score: -13.63847541809082 	 ['covid-19', 'disease_disease', 'he'] 
Score: -13.865299224853516 	 ['covid-19', 'disease_disease', 'same'] 
Score: -14.041131973266602 	 ['covid-19', 'disease_disease', 'sars cov infection'] 
Score: -14.156503677368164 	 ['covid-19', 'disease_disease', 'lupus erythematosus'] 
Score: -14.17286205291748 	 ['covid-19', 'disease_disease', 'covid'] 
Score: -14.210190773010254 	 ['covid-19', 'disease_disease', 'obesity'] 
Score: -14.258918762207031 	 ['covid-19', 'disease_disease', 'copd'] 
Score: -14.288290023803711 	 ['covid-19', 'disease_dise

In [31]:
for subset in ALL:
  


[{'subset_1':                                                triple      score
  0   [covid-19, disease_treat_procedure, pneumonic ... -17.668777
  1             [covid-19, disease_treat_procedure, ib] -17.788727
  2        [covid-19, disease_treat_procedure, vaccine] -17.793913
  3     [covid-19, disease_treat_procedure, astrovirus] -17.842379
  4           [covid-19, disease_treat_procedure, h5n1] -17.848053
  5   [covid-19, disease_treat_procedure, influenza ... -17.915321
  6   [covid-19, disease_treat_procedure, hmpv infec... -17.937637
  7             [covid-19, disease_treat_procedure, dt] -18.062168
  8   [covid-19, disease_treat_procedure, influenza ... -18.085270
  9    [covid-19, disease_treat_procedure, recruitment] -18.177042
  10       [disease, treat_procedure_disease, covid-19] -15.785202
  11     [community, treat_procedure_disease, covid-19] -15.839444
  12          [lung, treat_procedure_disease, covid-19] -15.890826
  13          [ards, treat_procedure_disease, covi

# QA EVALUATION